In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import nltk

In [2]:
# Cargo los datos de las películas\n",
df_movies = pd.read_csv("../data/launch/movies.csv")
df_genres = pd.read_csv("../data/launch/genres.csv")
df_movie_genres = pd.read_csv("../data/launch/movie_genres.csv")

### Realizamos Join y Limpieza de datos

In [3]:
# Combinar los DataFrames df_movie_genres y df_genres para añadir nombres de géneros a cada película.
df_movie_genres_2 = pd.merge(df_movie_genres, df_genres[["genre_id", "name"]], on="genre_id", how="left")

# Paso 1: Agrupar los géneros por id_movie y combinarlos en listas
# Utilizar el método groupby para agrupar los géneros por el ID de la película y luego unir los nombres de los géneros en una lista.
grouped_genres = df_movie_genres_2.groupby("id")["name"].agg(" ".join).reset_index()

# Paso 2: Realizar el join entre df_movies y grouped_genres
# Combinar el DataFrame df_movies con el DataFrame grouped_genres utilizando el ID de la película como clave de combinación.
df_movies = pd.merge(df_movies, grouped_genres, on="id", how="left")

# Cambiar el nombre de la columna "name" a "genres_name"
# Renombrar la columna "name" del DataFrame df_movies a "genres_name" para hacerla más descriptiva.
df_movies = df_movies.rename(columns={"name": "genres_name"})

# Cambiar el tipo de columna a string para la columna "genres_name"
# Cambiar el tipo de datos de la columna "genres_name" del DataFrame df_movies a tipo "string".
df_movies = df_movies.astype({"genres_name": "string"})

# Reemplazar los valores faltantes con "No Genre" en la columna "genres_name"
# Rellenar los valores faltantes en la columna "genres_name" del DataFrame df_movies con el texto "No Genre".
df_movies = df_movies.fillna({"genres_name": "No Genre"})

# Reemplazar todas las instancias de "," con "" en la columna "overview"
# Utilizar el método str.replace() para eliminar todas las comas "," en la columna "overview" del DataFrame df_movies.
df_movies["overview"] = df_movies["overview"].str.replace(",", "", case=False, regex=False)

In [4]:
df_movies.head(1)

,id,title,original_language,overview,tagline,status,runtime,release_date,release_year,budget,revenue,return,popularity,vote_average,vote_count,collection_id,has_collection,genres_name
0,2,Ariel,fi,Taisto Kasurinen is a Finnish coal miner whose...,NaN,Released,69,1988-10-21,1988,0.0,0.0,0.0,3.86,7.1,44,NaN,0,Drama Crime


In [5]:
df_movies = df_movies[["overview", "popularity","title", "genres_name"]]

In [6]:
df_movies.head(1)

,overview,popularity,title,genres_name
0,Taisto Kasurinen is a Finnish coal miner whose...,3.86,Ariel,Drama Crime


In [7]:
df_ML = df_movies.copy()

In [8]:
df_ML.shape

(45349, 4)

In [9]:
df_ML = df_ML.dropna().reset_index(drop=True)

In [10]:
df_ML.shape

(44408, 4)

In [11]:
df_ML=df_ML.head(100)

# Prueba de Modelo de Recomendación ML TF-IDF # 2
### Primero se buscan titulos completamente similares, luego similitud de todos los generos, luego similitud de descripción y por último puntaje de popularidad este es un enfoqiue que consume menos recursos que el modelo # 1

In [12]:
# Definir una función para preprocesar texto convirtiéndolo a minúsculas
def preprocess_text(text):
    return text.lower()

# Definir una función para calcular la similitud del coseno entre matrices
def calculate_cosine_similarity(matrix):
    return cosine_similarity(matrix)

# Definir una función para calcular la similitud de Jaccard entre dos listas de géneros
def calculate_jaccard_similarity(genre_list1, genre_list2):
    intersection = len(set(genre_list1).intersection(genre_list2))
    union = len(set(genre_list1).union(genre_list2))
    return intersection / union if union != 0 else 0.0

# Definir una función para precomputar una matriz a partir de una columna usando un vectorizador
def precompute_matrix(column, vectorizer):
    matrix = vectorizer.fit_transform(column)
    return matrix

# Definir una función para obtener películas similares basadas en una matriz de similitud y un índice de referencia
def get_similar_movies(similarity_matrix, reference_index):
    similar_movies = list(enumerate(similarity_matrix[reference_index]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    return similar_movies

# Definir una función para obtener las películas más populares
def get_top_popular_movies(df_ML):
    df_ML_sorted = df_ML.sort_values(by="popularity", ascending=False)
    top_popular_movies = df_ML_sorted.iloc[:5]
    return top_popular_movies

# Definir una función para recomendar películas similares
def recommend_movies(title, df_ML, title_to_index):
    movie_index = title_to_index.get(title)
    if movie_index is None:
        return ["No se encontró la película en la base de datos."]
    
    similar_movies_title = get_similar_movies(titulo_similarity, movie_index)
    similar_movies_genre = [(i, calculate_jaccard_similarity(df_ML["genres_clean"][movie_index], row["genres_clean"]))
                            for i, row in df_ML.iterrows()]
    similar_movies_description = get_similar_movies(descripcion_similarity, movie_index)
    
    all_similarities = [(idx, (sim_title + sim_genre + sim_description) / 3.0)
                        for (idx, sim_title), (_, sim_genre), (_, sim_description)
                        in zip(similar_movies_title, similar_movies_genre, similar_movies_description)]
    all_similarities = sorted(all_similarities, key=lambda x: x[1], reverse=True)
    
    recommended_movies = []
    for idx, _ in all_similarities:
        if idx != movie_index and len(recommended_movies) < 5:
            recommended_movies.append(df_ML.iloc[idx]["title"])
    
    return recommended_movies

# Cargar los datos y realizar preprocesamiento
# (Aquí falta el código para cargar los datos y crear el DataFrame df_ML)

# Aplicar preprocesamiento a las columnas "title", "genres_name" y "overview"
df_ML["title_clean"] = df_ML["title"].apply(preprocess_text)
df_ML["genres_clean"] = df_ML["genres_name"].apply(lambda x: [genre.lower() for genre in x.split()])
df_ML["overview_clean"] = df_ML["overview"].apply(preprocess_text)

# Preparar matrices y cálculos de similitud usando vectorizadores
count_vectorizer = CountVectorizer(stop_words=nltk.corpus.stopwords.words("english"))
titulo_matrix = precompute_matrix(df_ML["title_clean"], count_vectorizer)
titulo_similarity = calculate_cosine_similarity(titulo_matrix)

descripcion_vectorizer = TfidfVectorizer(stop_words=nltk.corpus.stopwords.words("english"))
descripcion_matrix = precompute_matrix(df_ML["overview_clean"], descripcion_vectorizer)
descripcion_similarity = calculate_cosine_similarity(descripcion_matrix)

# Crear un diccionario de indexación por título para uso en la función de recomendación
title_to_index = {title: idx for idx, title in enumerate(df_ML["title"])}

In [13]:
# Buscamos la Película de preferencia
recommended_movies = recommend_movies("Ariel", df_ML, title_to_index)
print(recommended_movies)

['Taxi Driver', 'Dancer in the Dark', 'A History of Violence', 'Absolute Power', 'Paradise Now']


# Creación de una función para el pregrabado general de las recomendaciones
### Este modelo consume muchos recursos de memoria por esto hemos optado por usar Microsoft Fabric quien nos da un poco más de recursos y obtener 15000 ejemplares de recomendaciones.

In [14]:
# Crear una lista para almacenar los datos
result_data = []

# Recorrer cada título en el DataFrame
for title in df_ML["title"]:
    recommended_movies = recommend_movies(title, df_ML, title_to_index)
    recommended_movies_str = ", ".join(recommended_movies)
    
    result_data.append({"movie_title": title, "recommended_titles": recommended_movies_str})

# Crear un DataFrame a partir de la lista de datos
result_df = pd.DataFrame(result_data, columns=["movie_title", "recommended_titles"])

# Guardar el DataFrame en un archivo CSV
result_df.to_csv("../data/ML/recomendaciones_peliculas_ML2_100.csv", index=False)

In [15]:
def get_recommendations_for_title(title):
    # Cargar el DataFrame precalculado desde el archivo CSV
    # Cargamos un DataFrame llamado "preview_recommendations" a partir del archivo CSV que contiene las recomendaciones precalculadas.
    preview_recommendations = pd.read_csv("../data/ML/recomendaciones_peliculas_ML2_100.csv")

    # Filtrar las recomendaciones para el título dado
    recommendations = preview_recommendations[preview_recommendations["movie_title"] == title]
    
    # Verificar si hay recomendaciones para el título dado
    if not recommendations.empty:
        # Si hay recomendaciones, obtener los títulos recomendados y dividirlos en una lista utilizando la coma como separador
        recommended_titles = recommendations.iloc[0]["recommended_titles"]
        return recommended_titles.split(", ")
    else:
        # Si no se encontraron recomendaciones para el título dado, devolver un mensaje de error
        return ["Película no encontrada en la Base de Datos."]

In [16]:
# Buscamos la Película de preferencia
title_to_search = "Ariel"
recommended_titles = get_recommendations_for_title(title_to_search)
print(recommended_titles)

['Taxi Driver', 'Dancer in the Dark', 'A History of Violence', 'Absolute Power', 'Paradise Now']
